In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow import keras
from datetime import datetime

In [2]:
# Load data
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
seed = 123
batch_size = 1024
train_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='validation', seed=seed)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

84131840/84125825 [==============================] - 4s 0us/step
Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [3]:
# Preprocessing
# Create a custom standardization function to strip HTML break
# tags ’<br />’.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation), ' ')
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100
# Use the text vectorization layer to normalize, split, and map
# strings to integers. Note that the layer uses the custom
# standardization defined above.
# Set maximum_sequence length as all samples are not of the same
# length.
vectorize_layer = TextVectorization(
                              standardize=custom_standardization,
                              max_tokens=vocab_size,
                              output_mode='int',
                              output_sequence_length=sequence_length)
# Make a text-only dataset (no labels) and call adapt to build
# the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [20]:
embedding_dim = 16

#Generate a model
model = keras.Sequential([vectorize_layer,
                          keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
                          keras.layers.GlobalAveragePooling1D(), # To deal with the fact that sentences do not have the same length
                          keras.layers.Dense(16, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid')])

In [21]:
model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['binary_accuracy'])

In [22]:
model.fit(train_ds, batch_size=64, epochs=30, validation_data=val_ds)

Epoch 1/40
20/20 [==============================] - 3s 125ms/step - loss: 0.6917 - binary_accuracy: 0.5687 - val_loss: 0.6835 - val_binary_accuracy: 0.7100
Epoch 2/40
20/20 [==============================] - 2s 116ms/step - loss: 0.6789 - binary_accuracy: 0.7250 - val_loss: 0.6631 - val_binary_accuracy: 0.7288
Epoch 3/40
20/20 [==============================] - 2s 114ms/step - loss: 0.6544 - binary_accuracy: 0.7428 - val_loss: 0.6310 - val_binary_accuracy: 0.7508
Epoch 4/40
20/20 [==============================] - 2s 116ms/step - loss: 0.6158 - binary_accuracy: 0.7710 - val_loss: 0.5874 - val_binary_accuracy: 0.7682
Epoch 5/40
20/20 [==============================] - 2s 116ms/step - loss: 0.5645 - binary_accuracy: 0.7963 - val_loss: 0.5399 - val_binary_accuracy: 0.7862
Epoch 6/40
20/20 [==============================] - 2s 119ms/step - loss: 0.5090 - binary_accuracy: 0.8181 - val_loss: 0.4961 - val_binary_accuracy: 0.8016
Epoch 7/40
20/20 [==============================] - 2s 114ms/ste

KeyboardInterrupt: ignored